# Exploratory Data Analysis of Epicurious Scrape in a JSON file

This is an idealized workflow for Aaron Chen in looking at data science problems. It likely isn't the best path, nor has he rigidly applied or stuck to this ideal, but he wishes that he worked this way more frequently.

## Purpose: Work through some exploratory data analysis of the Epicurious scrape on stream. Try to write some functions to help process the data.

### Author: Aaron Chen

---

### If needed, run shell commands here

In [ ]:
# !python -m spacy download en_core_web_sm
!dvc pull -r origin
!dvc commit -f ../data.dvc

  0% Checkout|                                       |0/1 [00:00<?,     ?file/s]
!
Building data objects from ../data                    |0.00 [00:00,      ?obj/s]
Everything is up to date.                                                       
                                                                            

---

## External Resources

List out references or documentation that has helped you with this notebook

### Code
Regex Checker: https://regex101.com/

#### Scikit-learn
1. https://scikit-learn.org/stable/modules/decomposition.html#latent-dirichlet-allocation-lda


### Data

For this notebook, the data is stored in the repo base folder/data/raw

### Process

Are there steps or tutorials you are following? Those are things I try to list in Process

---

## Import necessary libraries

In [ ]:
# | hide
import project_path
from datetime import datetime
import dvc.api
import numpy as np
import pandas as pd
from sklearn.base import TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import spacy
import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS
from tqdm import tqdm
from typing import Dict
import unicodedata

# import local scripts
import src.dataframe_preprocessor as dfpp

---

## Define helper functions

My workflow is to try things with code cells, then when the code cells get messy and repetitive, to convert into helper functions that can be called.

When the helper functions are getting used a lot, it is usually better to convert them to scripts or classes that can be called/instantiated

---

## Define global variables 

**Remember to refactor these out, not ideal**

In [ ]:
# | hide
data_path = "data/recipes-en-201706/epicurious-recipes_m2.json"

repo_url = "https://dagshub.com/AaronWChen/MeaLeon"

---

## Running Commentary

1. I used numbered lists to keep track of things I noticed

### To Do

1. Try to determine consistency of nested data structures
   1. Is the photoData or number of things inside photoData the same from record to record
   2. What about for tag?

Data wasn't fully consistent but logic in helper function helped handle nulls

2. How to handle nulls?
   1. Author      Filled in with "Missing Author"
   2. Tag         Filled in with "Missing Cuisine"
3. ~~Convert pubDate to actual timestamp~~  
4. ~~Convert ScrapeDate to actual timestamp~~
   1. This was ignored as the datestamp was not useful (generally within minutes of the origin of UNIX time)
   
**5. Append new columns for relevant nested structures and unfold them**

6. Determine actual types of `ingredients` and `prepSteps`
7. Continue working through test example of single recipe to feed into spaCy and then sklearn.feature_extraction.text stack
8. Will need to remove numbers, punctuation

---

## Importing and viewing the data as a dataframe

In [ ]:
# with dvc.api.open(data_path, repo=repo_url) as f:
data = dvc.api.read(path=data_path, repo=repo_url, rev="dev", remote="origin-s3")
epic_dataframe = pd.read_json(data)
epic_dataframe.head()

PathMissingError: The path 'data/recipes-en-201706/epicurious-recipes_m2.json' does not exist in the target repository 'https://dagshub.com/AaronWChen/MeaLeon' neither as a DVC output nor as a Git-tracked file.

In [ ]:
epic_dataframe["type"].value_counts()

In [ ]:
epic_dataframe.shape

In [ ]:
epic_dataframe.describe()

In [ ]:
epic_dataframe.info()

In [ ]:
epic_dataframe["aggregateRating"].value_counts()

Columns:

    Index

    ID: string

    dek: appears to be description of the recipe, string

    hed: Appears to be title, string

    pubDate: appears to be publication date, may need to reformat to datetime objects

    author: appears that each record contains an array (list), inside each list is a dictionary with 'name' as the key and author name as the value. Notably, not a unique identifier for the value. Because the data is technically nested, may need to extract and transform and add columns to dataframe

    type: string, but all the values are exactly the same and they are all in the category of "recipe". Drop column

    url: Appears to be a long string leading to where the recipe can be found on Epicurious's website

    photoData: nested structure, inside each record is a dictionary

    tag: each record contains a dictionary. may need to extract and transform and add columns to dataframe

    aggregateRating: float, let's say it's out of 4.0

    ingredients: appears to be a list, does look like a list of strings

    prepSteps: appears to be a list, does look like a list of strings

    reviewsCount: int

    willMakeAgainPct: integer	
    
    dateCrawled: appears to be a unix timestamp

Let's take a look at the possibly problematic columns and see if the data structures make sense or how we can approach transforming them into new columns for the dataset

In [ ]:
epic_dataframe.loc[0]

In [ ]:
epic_dataframe.loc[0]["photoData"]

It looks like photoData contains:
    1. photo ID, string
    2. photo filename, string
    3. photo caption, string
    4. photo credit, string
    5. promoTitle, string
    6. title, string
       1. caption, promoTitle, and title could be all the same
    7. orientation, string
    8. restrictCropping: boolean

Of these, maybe we should keep
id => photoID
filename => photoFilename
caption => photoCaption
credit => photoCredit


In [ ]:
epic_dataframe.loc[0]["tag"]

In [ ]:
epic_dataframe.loc[100]["tag"]

In [ ]:
epic_dataframe.loc[10]["tag"]

In [ ]:
epic_dataframe.loc[1]["tag"]

In [ ]:
epic_dataframe.loc[1]["ingredients"]

In [ ]:
type(epic_dataframe.loc[1]["ingredients"])

Save this aside for CountVectorization/Natural Language Processing later, continue doing feature exploration

In [ ]:
epic_dataframe.loc[1]["dek"]

In [ ]:
epic_dataframe["tag"][2]

In [ ]:
epic_dataframe["cuisine_name"] = epic_dataframe["tag"].apply(
    lambda x: x["name"]
    if not pd.isna(x) and x["category"] == "cuisine"
    else "Cuisine Missing"
)

In [ ]:
epic_dataframe["cuisine_name"].head()

In [ ]:
epic_dataframe["cuisine_name"][0]

In [ ]:
print(epic_dataframe.shape)
print(epic_dataframe[epic_dataframe["cuisine_name"] != "Missing"].shape)

In [ ]:
epic_dataframe[epic_dataframe["cuisine_name"] == "Cuisine Missing"]

In [ ]:
# this lambda function goes into the photo data column and extracts just the filename from the dictionary
epic_dataframe["photo_filename"] = epic_dataframe["photoData"].apply(
    lambda x: x["filename"] if not pd.isna(x) else "Missing photo"
)

# This lambda function goes into the photo data column and extracts just the photo credit from the dictionary
epic_dataframe["photo_credit"] = epic_dataframe["photoData"].apply(
    lambda x: x["credit"] if not pd.isna(x) else "Missing credit"
)

In [ ]:
epic_dataframe.head()

In [ ]:
# This lambda function cleans up the column and adds a new column dataframe
epic_dataframe["author_name"] = epic_dataframe["author"].apply(
    lambda x: x[0]["name"] if not pd.isna(x) else "Missing author name"
)

This function did not work, what happened?

In [ ]:
first_five_epic = epic_dataframe.head()

In [ ]:
first_five_epic

In [ ]:
first_five_epic.iloc[0]["author"][0]

In [ ]:
first_five_epic.iloc[1]["author"][0]

In [ ]:
first_five_epic["author"].apply(lambda x: x[0]["name"] if x else "Missing author name")

This lambda function now works enough! It goes into author column and extracts the author as long as the record isn't an empty list. This can be refactored into a helper function. But we need to apply to the whole dataset

In [ ]:
epic_dataframe["author_name"] = epic_dataframe["author"].apply(
    lambda x: x[0]["name"] if x else "Missing author name"
)

In [ ]:
epic_dataframe

## Let's add a feature to fix the datetimes

In [ ]:
test_pubdate_array = epic_dataframe["pubDate"][0:5]
test_pubdate_array

In [ ]:
print(type(test_pubdate_array[0][:10]))

In [ ]:
test_pubdate_array[0][:10]

In [ ]:
epic_dataframe["publication_date"] = epic_dataframe["date_published"].apply(
    lambda x: datetime.strptime(x[:10], "%Y-%m-%d")
)

In [ ]:
epic_dataframe["publication_date"]

In [ ]:
epic_dataframe["publication_date_todt"] = pd.to_datetime(
    epic_dataframe["pubDate"], infer_datetime_format=True
)

In [ ]:
epic_dataframe["publication_date_todt"]

Don't need the apply with lambda function anymore because to_datetime succesfully resolved the odd string

In [ ]:
epic_dataframe["date_scraped"] = pd.to_datetime(
    epic_dataframe["dateCrawled"], infer_datetime_format=True
)
epic_dataframe["date_scraped"]

In [ ]:
epic_dataframe["date_scraped"].describe()

In [ ]:
epic_dataframe["tag"]

Based on the timestamps, it seems like we can drop the crawled/scraped column because the values don't really make sense and would not help

### Next Steps
- Refactor datetime processing into functions
- Consider deploying as a pd.pipe()


### Checking how to get the correct path inside the project_path script

In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
sys.path

In [ ]:
module_path

In [ ]:
os.pardir